# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Мы имеем пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Необходимо построить модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Также проанализировать возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [83]:
pip install phik

Note: you may need to restart the kernel to use updated packages.


In [84]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import phik

In [85]:
BUDGET = 10_000_000_000
REVENUE = 450_000
POINTS = 500
BEST_POINTS = 200

In [86]:
data0 = pd.read_csv('/datasets/geo_data_0.csv') 
data0.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [87]:
data1 = pd.read_csv('/datasets/geo_data_1.csv')
data1.head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [88]:
data2 = pd.read_csv('/datasets/geo_data_2.csv')
data2.head()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


In [89]:
data0.info(), data1.info(), data2.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null 

(None, None, None)

In [90]:
data0.isna().sum(), data1.isna().sum(), data2.isna().sum() 

(id         0
 f0         0
 f1         0
 f2         0
 product    0
 dtype: int64,
 id         0
 f0         0
 f1         0
 f2         0
 product    0
 dtype: int64,
 id         0
 f0         0
 f1         0
 f2         0
 product    0
 dtype: int64)

In [91]:
data0.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


In [92]:
data1.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408


In [93]:
data2.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


In [94]:
data0=data0.drop('id',axis=1) 
data1=data1.drop('id', axis=1)
data2=data2.drop('id', axis=1)

Для изучения нам представлены 3 датасеты с данными по трём регионам, пропусков в данных нет, в каждом регионе 100 000 наблюдений, столбец с уникальным индификатором скважин решила удалить, так как он неинформативен, и будет только мешать при предсказании модели. Признаками выделем по каждому региону - F0, F1, F2. Целевой признак - 'product' ( объём запасов в скважине). Категориальных признаков нет, только численные с типом float.

## Обучение и проверка модели

In [ ]:
def data0_model (f1, f2, f3, product):
    data0_train, data0_valid=train_test_split(data0, test_size=0.25, random_state=12345)
    features0_train = data0_train.drop(['product'], axis=1) 
    target0_train = data0_train['product']
    features0_valid = data0_valid.drop(['product'], axis=1)
    target0_valid = data0_valid['product']
    model0 =LinearRegression() 
    model0.fit(features0_train, target0_train) 
    predictions0_valid = model0.predict(features0_valid)
    

In [95]:
data0_train, data0_valid=train_test_split(data0, test_size=0.25, random_state=12345) 
data1_train, data1_valid=train_test_split(data1, test_size=0.25, random_state=12345)
data2_train, data2_valid=train_test_split(data2, test_size=0.25, random_state=12345)

In [96]:
features0_train = data0_train.drop(['product'], axis=1) 
target0_train = data0_train['product']
features0_valid = data0_valid.drop(['product'], axis=1)
target0_valid = data0_valid['product']
features1_train = data1_train.drop(['product'], axis=1)
target1_train = data1_train['product']
features1_valid = data1_valid.drop(['product'], axis=1)
target1_valid = data1_valid['product']
features2_train = data2_train.drop(['product'], axis=1)
target2_train = data2_train['product']
features2_valid = data2_valid.drop(['product'], axis=1)
target2_valid = data2_valid['product']

In [97]:
model0 =LinearRegression() 
model0.fit(features0_train, target0_train) 
predictions0_valid = model0.predict(features0_valid)
model1 =LinearRegression() 
model1.fit(features1_train, target1_train) 
predictions1_valid = model1.predict(features1_valid)
model2 =LinearRegression() 
model2.fit(features2_train, target2_train) 
predictions2_valid = model2.predict(features2_valid)

In [98]:
print('RMSE0:', mean_squared_error(target0_valid, predictions0_valid)**0.5) 
print('Средний запас сырья для региона 0:', target0_train.mean())
print('RMSE1:', mean_squared_error(target1_valid, predictions1_valid)**0.5)
print('Средний запас сырья для региона 1:', target1_train.mean())
print('RMSE2:', mean_squared_error(target2_valid, predictions2_valid)**0.5)
print('Средний запас сырья для региона 2:', target2_train.mean())

RMSE0: 37.5794217150813
Средний запас сырья для региона 0: 92.64046775305692
RMSE1: 0.893099286775617
Средний запас сырья для региона 1: 68.85895465854666
RMSE2: 40.02970873393434
Средний запас сырья для региона 2: 95.03858906371522


Поскольку значение RMSE для второго региона является самым низким, я бы выбрала  в качестве лучшей модели для прогнозирования, хотя и средний запас сырья в нём самый низкий. Для третьего региона региона среднеквадратическая ошибка самая высокая.

## Подготовка к расчёту прибыли

In [99]:
BARREL = (BUDGET/REVENUE)/BEST_POINTS 
print(f'Точка безубыточности: {int(BARREL)} тыс. баррелей')

Точка безубыточности: 111 тыс. баррелей


In [100]:
mean0 = data0['product'].mean()
mean1 = data1['product'].mean()
mean2 = data2['product'].mean()
print(f' Средний запас сырья в первом регионе: {int(mean0)} тыс.баррелей')
print(f' Средний запас сырья во втором регионе: {int(mean1)} тыс.баррелей')
print(f' Средний запас сырья в третьем регионе: {int(mean2)} тыс.баррелей')

 Средний запас сырья в первом регионе: 92 тыс.баррелей
 Средний запас сырья во втором регионе: 68 тыс.баррелей
 Средний запас сырья в третьем регионе: 95 тыс.баррелей


Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины. Общий бюджет для разработки скважин 10 млрд. рублей, а доход с тысячи баррелей нефти составляет 450 000 рублей. Бюджет выделяется на разработку 200 скважин. Получается, что объём сырья должен быть не менее 111 тыс. баррелей. Средний реальный запас в каждом регионе меньше, чем рассчитанный показатель безубыточности: в первом регионе на 19 тыс. баррелей, во втором-на 43 тыс., в третьем -на 16 тыс. баррелей

## Расчёт прибыли и рисков 

In [125]:
pred0_valid = pd.Series(predictions0_valid, index = target0_valid.index) 
def income (target0_valid, pred0_valid, count):
    probs_sorted = pred0_valid.sort_values(ascending=False)
    selected = target0_valid[probs_sorted.index][:count]
    return (revenue* selected.sum() - budget)
state = np.random.RandomState(12345)
values = []
for i in range(1000):
    target_subsample = target0_valid.sample(POINTS, replace = True, random_state=state)
    probs_subsample = pred0_valid[target_subsample.index]
    target_subsample = target_subsample.reset_index(drop = True)
    probs_subsample = probs_subsample.reset_index(drop = True)
    values.append(income(target_subsample, probs_subsample, BEST_POINTS))

values = pd.Series(values)
lower = values.quantile(0.025)
upper = values.quantile(0.975)
mean = values.mean()
risk = (sum(values < 0)/values.count())*100
print(f'Средняя прибыль первого региона: {mean.astype(int)//1000_000} млн рублей' )
print(f'2,5%-квантиль: {lower.astype(int)//1000_000} млн рублей')
print(f'97.5%-квантиль: {upper.astype(int)//1000_000} млн рублей')
print(f'Процент риска {risk} %')

Средняя прибыль: 396 млн рублей
2,5%-квантиль: -112 млн рублей
97.5%-квантиль: 909 млн рублей
Процент риска 6.9 %


In [123]:
pred1_valid = pd.Series(predictions1_valid, index = target1_valid.index) 
def income (target1_valid, pred1_valid, count):
    probs_sorted = pred1_valid.sort_values(ascending=False)
    selected = target1_valid[probs_sorted.index][:count]
    return (revenue* selected.sum() - budget)
state = np.random.RandomState(12345)
values = []
for i in range(1000):
    target_subsample = target1_valid.sample(POINTS, replace = True, random_state=state)
    probs_subsample = pred1_valid[target_subsample.index]
    target_subsample = target_subsample.reset_index(drop = True)
    probs_subsample = probs_subsample.reset_index(drop = True)
    values.append(income(target_subsample, probs_subsample, BEST_POINTS))

values = pd.Series(values)
lower = values.quantile(0.025)
upper = values.quantile(0.975)
mean = values.mean()
risk = (sum(values < 0)/values.count())*100
print(f'Средняя прибыль второго региона: {mean.astype(int)//1000_000} млн рублей' )
print(f'2,5%-квантиль: {lower.astype(int)//1000_000} млн рублей')
print(f'97.5%-квантиль: {upper.astype(int)//1000_000} млн рублей')
print(f'Процент риска {risk} %')

Средняя прибыль: 456 млн рублей
2,5%-квантиль: 33 млн рублей
97.5%-квантиль: 852 млн рублей
Процент риска 1.5 %


In [121]:
pred2_valid = pd.Series(predictions2_valid, index = target2_valid.index) 
def income (target2_valid, pred2_valid, count):
    probs_sorted = pred2_valid.sort_values(ascending=False)
    selected = target2_valid[probs_sorted.index][:count]
    return (REVENUE* selected.sum() - BUDGET)
state = np.random.RandomState(12345)
values = []
for i in range(1000):
    target_subsample = target2_valid.sample(POINTS, replace = True, random_state=state)
    probs_subsample = pred2_valid[target_subsample.index]
    target_subsample = target_subsample.reset_index(drop = True)
    probs_subsample = probs_subsample.reset_index(drop = True)
    values.append(income(target_subsample, probs_subsample, BEST_POINTS))

values = pd.Series(values)
lower = values.quantile(0.025)
upper = values.quantile(0.975)
mean = values.mean()
risk = (sum(values < 0)/values.count())*100
print(f'Средняя прибыль третьего региона: {mean.astype(int)//1000_000} млн рублей' )
print(f'2,5%-квантиль: {lower.astype(int)//1000_000} млн рублей')
print(f'97.5%-квантиль: {upper.astype(int)//1000_000} млн рублей')
print(f'Процент риска {risk} %')

Средняя прибыль: 404 млн рублей
2,5%-квантиль: -164 млн рублей
97.5%-квантиль: 950 млн рублей
Процент риска 7.6 %


<div class="alert alert-info"> ВЫВОД: У нас даны данные по трём регионам, в каждом регионе данные по 100 000 скважинам (с объёмом запасов нефти в каждой скважине). Для того чтобы новая скважина была прибыльной запас нефти в ней должен быть более 111 тыс баррелей. По данным вы видим, что такие скважины есть в каждом регионе, однако в среднем по каждому региону запас сырья меньше чем точка безубыточности. Ддя выбора лучшего региона мы случайным образом выбрали 500 скважин из них выбрали 200 лучших для разработки. Я предполагаю, что самым подходящим регионом для разработки будет второй регион, так как результаты исследования показали, что во втором регионе самый большой средний показатель прибыли (456 млн рублей) при минимальном риске (1,5%).